# LDA for news headlines

My revised code for 

- https://github.com/susanli2016/NLP-with-Python/blob/master/LDA_news_headlines.ipynb
- https://towardsdatascience.com/topic-modeling-and-latent-dirichlet-allocation-in-python-9bf156893c24



`abcnews-small.csv` is a ~10k subset of the original 1 million dataset that can be downloaded at https://www.kaggle.com/therohk/million-headlines, which is about 60m unzipped. 

Running the whole dataset on a 2017 MacBook Pro:

- the preprocessing step is about 3 minutes: `processed_docs = documents['headline_text'].map(preprocess)`
- the training step using bag of words is about 5min 46s: `lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)`
- the training step using tf-idf is about 9min 21s: `lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)`

In [44]:
import pandas as pd

# uncomment the following line if you want to try the full dataset
#data = pd.read_csv('data/abcnews-date-text.csv', error_bad_lines=False);

# read a subset of the full dataset
data = pd.read_csv('data/abcnews-small.csv', error_bad_lines=False);

data_text = data[['headline_text']]
data_text['index'] = data_text.index
documents = data_text

In [45]:
len(documents)

10001

In [46]:
documents[:5]

,headline_text,index
0,report highlights container terminal potential,0
1,mud crab business on the move,1
2,sporting task force planning begins,2
3,ama airs hospital reform concerns,3
4,health service speaks out over patient death,4


### Data Preprocessing

In [47]:
import gensim
from gensim.utils import simple_preprocess
from gensim.parsing.preprocessing import STOPWORDS
from nltk.stem import WordNetLemmatizer, SnowballStemmer
from nltk.stem.porter import *
import numpy as np
np.random.seed(42)

In [48]:
import nltk
nltk.download('wordnet')

[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/harrywang/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

#### Lemmatize example

In [49]:
print(WordNetLemmatizer().lemmatize('went', pos='v'))

go


#### Stemmer Example

In [50]:
stemmer = SnowballStemmer('english')
original_words = ['caresses', 'flies', 'dies', 'mules', 'denied','died', 'agreed', 'owned', 
           'humbled', 'sized','meeting', 'stating', 'siezing', 'itemization','sensational', 
           'traditional', 'reference', 'colonizer','plotted']
singles = [stemmer.stem(plural) for plural in original_words]
pd.DataFrame(data = {'original word': original_words, 'stemmed': singles})

,original word,stemmed
0,caresses,caress
1,flies,fli
2,dies,die
3,mules,mule
4,denied,deni
5,died,die
6,agreed,agre
7,owned,own
8,humbled,humbl
9,sized,size


In [51]:
def lemmatize_stemming(text):
    return stemmer.stem(WordNetLemmatizer().lemmatize(text, pos='v'))

def preprocess(text):
    result = []
    for token in gensim.utils.simple_preprocess(text):
        if token not in gensim.parsing.preprocessing.STOPWORDS and len(token) > 3:
            result.append(lemmatize_stemming(token))
    return result

In [52]:
doc_sample = documents[documents['index'] == 4310].values[0][0]

print('original document: ')
words = []
for word in doc_sample.split(' '):
    words.append(word)
print(words)
print('\n\n tokenized and lemmatized document: ')
print(preprocess(doc_sample))

original document: 
[&#39;charity&#39;, &#39;in&#39;, &#39;demand&#39;, &#39;from&#39;, &#39;sacked&#39;, &#39;meatworkers&#39;]


 tokenized and lemmatized document: 
[&#39;chariti&#39;, &#39;demand&#39;, &#39;sack&#39;, &#39;meatwork&#39;]


In [53]:
%%time
# for the 1 million dataset, wall time: 2min 59s
processed_docs = documents['headline_text'].map(preprocess)

CPU times: user 1.58 s, sys: 107 ms, total: 1.69 s
Wall time: 2.65 s


In [54]:
processed_docs[:10]

0    [report, highlight, contain, termin, potenti]
1                                     [crab, busi]
2                 [sport, task, forc, plan, begin]
3                   [air, hospit, reform, concern]
4          [health, servic, speak, patient, death]
5                              [hors, ride, enact]
6                 [plan, intersect, revamp, begin]
7                      [stormer, slaughter, shark]
8               [england, deserv, favourit, oneil]
9      [firefight, continu, contain, effort, near]
Name: headline_text, dtype: object

### Bag of words on the dataset

In [55]:
dictionary = gensim.corpora.Dictionary(processed_docs)

In [56]:
count = 0
for k, v in dictionary.iteritems():
    print(k, v)
    count += 1
    if count > 10:
        break

0 contain
1 highlight
2 potenti
3 report
4 termin
5 busi
6 crab
7 begin
8 forc
9 plan
10 sport


In [57]:
dictionary.filter_extremes(no_below=15, no_above=0.5, keep_n=100000)

In [58]:
bow_corpus = [dictionary.doc2bow(doc) for doc in processed_docs]
bow_corpus[4310]

[(277, 1), (435, 1)]

In [59]:
bow_doc_4310 = bow_corpus[4310]

for i in range(len(bow_doc_4310)):
    print("Word {} (\"{}\") appears {} time.".format(bow_doc_4310[i][0], 
                                                     dictionary[bow_doc_4310[i][0]], 
                                                     bow_doc_4310[i][1]))

Word 277 (&quot;demand&quot;) appears 1 time.
Word 435 (&quot;sack&quot;) appears 1 time.


### TF-IDF

In [60]:
from gensim import corpora, models

tfidf = models.TfidfModel(bow_corpus)

In [61]:
corpus_tfidf = tfidf[bow_corpus]

In [62]:
from pprint import pprint

for doc in corpus_tfidf:
    pprint(doc)
    break

[(0, 0.802436033281265), (1, 0.5967381439893285)]


### Running LDA using Bag of Words

In [63]:
%%time
lda_model = gensim.models.LdaMulticore(bow_corpus, num_topics=10, id2word=dictionary, passes=2, workers=2)

CPU times: user 2.22 s, sys: 631 ms, total: 2.85 s
Wall time: 5.83 s


In [64]:
for idx, topic in lda_model.print_topics(-1):
    print('Topic: {} \nWords: {}'.format(idx, topic))

Topic: 0 
Words: 0.037*&quot;interview&quot; + 0.035*&quot;polic&quot; + 0.020*&quot;home&quot; + 0.015*&quot;hous&quot; + 0.014*&quot;school&quot; + 0.013*&quot;brisban&quot; + 0.012*&quot;question&quot; + 0.011*&quot;plead&quot; + 0.010*&quot;mark&quot; + 0.010*&quot;sydney&quot;
Topic: 1 
Words: 0.022*&quot;kill&quot; + 0.020*&quot;protest&quot; + 0.019*&quot;polic&quot; + 0.019*&quot;chang&quot; + 0.015*&quot;budget&quot; + 0.014*&quot;trump&quot; + 0.013*&quot;island&quot; + 0.013*&quot;tasmanian&quot; + 0.013*&quot;speak&quot; + 0.011*&quot;urg&quot;
Topic: 2 
Words: 0.040*&quot;australia&quot; + 0.026*&quot;year&quot; + 0.019*&quot;elect&quot; + 0.016*&quot;servic&quot; + 0.015*&quot;bank&quot; + 0.014*&quot;council&quot; + 0.013*&quot;miss&quot; + 0.012*&quot;vote&quot; + 0.012*&quot;plan&quot; + 0.011*&quot;high&quot;
Topic: 3 
Words: 0.013*&quot;plan&quot; + 0.013*&quot;govern&quot; + 0.013*&quot;court&quot; + 0.013*&quot;farmer&quot; + 0.012*&quot;public&quot; + 0.012*&quot;

Cool! Can you distinguish different topics using the words in each topic and their corresponding weights

### Running LDA using TF-IDF

In [65]:
%%time
lda_model_tfidf = gensim.models.LdaMulticore(corpus_tfidf, num_topics=10, id2word=dictionary, passes=2, workers=4)

CPU times: user 1.81 s, sys: 341 ms, total: 2.15 s
Wall time: 4.6 s


In [66]:
for idx, topic in lda_model_tfidf.print_topics(-1):
    print('Topic: {} Word: {}'.format(idx, topic))

Topic: 0 Word: 0.020*&quot;win&quot; + 0.017*&quot;australian&quot; + 0.017*&quot;tasmania&quot; + 0.015*&quot;test&quot; + 0.013*&quot;research&quot; + 0.013*&quot;look&quot; + 0.012*&quot;cricket&quot; + 0.011*&quot;launch&quot; + 0.010*&quot;port&quot; + 0.010*&quot;food&quot;
Topic: 1 Word: 0.026*&quot;interview&quot; + 0.020*&quot;say&quot; + 0.019*&quot;woman&quot; + 0.016*&quot;charg&quot; + 0.015*&quot;final&quot; + 0.012*&quot;go&quot; + 0.012*&quot;perth&quot; + 0.009*&quot;polic&quot; + 0.009*&quot;australian&quot; + 0.008*&quot;indigen&quot;
Topic: 2 Word: 0.019*&quot;say&quot; + 0.016*&quot;market&quot; + 0.016*&quot;polic&quot; + 0.014*&quot;time&quot; + 0.013*&quot;news&quot; + 0.012*&quot;brisban&quot; + 0.011*&quot;work&quot; + 0.011*&quot;melbourn&quot; + 0.011*&quot;monday&quot; + 0.010*&quot;report&quot;
Topic: 3 Word: 0.021*&quot;elect&quot; + 0.017*&quot;miss&quot; + 0.016*&quot;plan&quot; + 0.012*&quot;talk&quot; + 0.012*&quot;centr&quot; + 0.012*&quot;leader&quo

In [67]:
processed_docs[4310]

[&#39;chariti&#39;, &#39;demand&#39;, &#39;sack&#39;, &#39;meatwork&#39;]

In [68]:
for index, score in sorted(lda_model[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model.print_topic(index, 10)))


Score: 0.6999604105949402	 
Topic: 0.018*&quot;dead&quot; + 0.013*&quot;go&quot; + 0.013*&quot;claim&quot; + 0.013*&quot;lose&quot; + 0.011*&quot;industri&quot; + 0.010*&quot;crash&quot; + 0.010*&quot;award&quot; + 0.009*&quot;region&quot; + 0.009*&quot;jail&quot; + 0.009*&quot;rule&quot;

Score: 0.033347394317388535	 
Topic: 0.013*&quot;plan&quot; + 0.013*&quot;govern&quot; + 0.013*&quot;court&quot; + 0.013*&quot;farmer&quot; + 0.012*&quot;public&quot; + 0.012*&quot;turnbul&quot; + 0.012*&quot;million&quot; + 0.011*&quot;campaign&quot; + 0.011*&quot;hospit&quot; + 0.011*&quot;climat&quot;

Score: 0.03334241360425949	 
Topic: 0.040*&quot;australia&quot; + 0.026*&quot;year&quot; + 0.019*&quot;elect&quot; + 0.016*&quot;servic&quot; + 0.015*&quot;bank&quot; + 0.014*&quot;council&quot; + 0.013*&quot;miss&quot; + 0.012*&quot;vote&quot; + 0.012*&quot;plan&quot; + 0.011*&quot;high&quot;

Score: 0.033337824046611786	 
Topic: 0.037*&quot;say&quot; + 0.026*&quot;attack&quot; + 0.016*&quot;crash

Our test document has the highest probability to be part of the topic on the top.

### Performance evaluation by classifying sample document using LDA TF-IDF model

In [69]:
for index, score in sorted(lda_model_tfidf[bow_corpus[4310]], key=lambda tup: -1*tup[1]):
    print("\nScore: {}\t \nTopic: {}".format(score, lda_model_tfidf.print_topic(index, 10)))


Score: 0.6999433040618896	 
Topic: 0.024*&quot;interview&quot; + 0.020*&quot;kill&quot; + 0.014*&quot;attack&quot; + 0.013*&quot;coast&quot; + 0.012*&quot;power&quot; + 0.011*&quot;releas&quot; + 0.011*&quot;say&quot; + 0.010*&quot;peopl&quot; + 0.010*&quot;action&quot; + 0.009*&quot;step&quot;

Score: 0.0333525687456131	 
Topic: 0.018*&quot;govern&quot; + 0.016*&quot;want&quot; + 0.012*&quot;group&quot; + 0.012*&quot;child&quot; + 0.010*&quot;fund&quot; + 0.010*&quot;abus&quot; + 0.010*&quot;train&quot; + 0.010*&quot;sentenc&quot; + 0.010*&quot;year&quot; + 0.009*&quot;call&quot;

Score: 0.03334299474954605	 
Topic: 0.026*&quot;interview&quot; + 0.020*&quot;say&quot; + 0.019*&quot;woman&quot; + 0.016*&quot;charg&quot; + 0.015*&quot;final&quot; + 0.012*&quot;go&quot; + 0.012*&quot;perth&quot; + 0.009*&quot;polic&quot; + 0.009*&quot;australian&quot; + 0.008*&quot;indigen&quot;

Score: 0.033339973539114	 
Topic: 0.020*&quot;trump&quot; + 0.018*&quot;death&quot; + 0.011*&quot;make&quot; 

Our test document has the highest probability to be part of the topic on the top.

### Testing model on unseen document

In [70]:
unseen_document = 'How a Pentagon deal became an identity crisis for Google'
bow_vector = dictionary.doc2bow(preprocess(unseen_document))

for index, score in sorted(lda_model[bow_vector], key=lambda tup: -1*tup[1]):
    print("Score: {}\t Topic: {}".format(score, lda_model.print_topic(index, 5)))

Score: 0.6999323964118958	 Topic: 0.020*&quot;work&quot; + 0.017*&quot;talk&quot; + 0.016*&quot;melbourn&quot; + 0.015*&quot;help&quot; + 0.015*&quot;arrest&quot;
Score: 0.03335041552782059	 Topic: 0.013*&quot;plan&quot; + 0.013*&quot;govern&quot; + 0.013*&quot;court&quot; + 0.013*&quot;farmer&quot; + 0.012*&quot;public&quot;
Score: 0.03334531560540199	 Topic: 0.037*&quot;say&quot; + 0.026*&quot;attack&quot; + 0.016*&quot;crash&quot; + 0.015*&quot;health&quot; + 0.014*&quot;state&quot;
Score: 0.0333448089659214	 Topic: 0.040*&quot;australia&quot; + 0.026*&quot;year&quot; + 0.019*&quot;elect&quot; + 0.016*&quot;servic&quot; + 0.015*&quot;bank&quot;
Score: 0.033340469002723694	 Topic: 0.022*&quot;open&quot; + 0.013*&quot;elect&quot; + 0.013*&quot;local&quot; + 0.012*&quot;make&quot; + 0.012*&quot;trump&quot;
Score: 0.03334016725420952	 Topic: 0.027*&quot;charg&quot; + 0.022*&quot;australian&quot; + 0.019*&quot;warn&quot; + 0.018*&quot;face&quot; + 0.017*&quot;polic&quot;
Score: 0.0333387

In [73]:
# random choose 10,000 about 1/100 from the whole dataset
import random

# no need to run this, skip by default
skip = True

if not skip:
    n = 1186018 # 1 million total 
    s = 10000 # random 10k
    skip = sorted(random.sample(range(n), n-s))
    data_small = pd.read_csv('data/abcnews-date-text.csv', skiprows=skip, error_bad_lines=False, names=['index', 'headline_text'])
    data_small.drop('index', axis=1, inplace=True)
    data_small.to_csv('data/abcnews-small.csv')